# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, Pvalb-IRES-Cre, Female, VISp, Static Gratings (SG) and Drifting Gratings (DG). 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'F') & \
                             (sessions.full_genotype.str.find('Pvalb-IRES-Cre') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISp' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
746083955,2019-10-03T00:00:00Z,726170935,brain_observatory_1.1,98.0,F,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,582,2216,6,"[VPM, TH, LGd, CA3, CA2, CA1, VISal, nan, grey..."
773418906,2019-10-03T00:00:00Z,757329624,brain_observatory_1.1,124.0,F,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,546,2232,6,"[PPT, NOT, SUB, ProS, CA1, VISam, nan, APN, DG..."


In [8]:
session_id = filtered_sessions.index.values[1]

session = cache.get_session_data(session_id)

Downloading:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

In [9]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

/Users/tulsipatel/.local/lib/python3.8/site-packages/allensdk/brain_observatory/ecephys/ecephys_project_api/ecephys_project_warehouse_api.py:291: FutureWarning: Conversion of the second argument of issubdtype from `bool` to `np.generic` is deprecated. In future, it will be treated as `np.bool_ == np.dtype(bool).type`.
  pv_is_bool = np.issubdtype(output["p_value_rf"].values[0], np.bool)


Total number of units: 546


,waveform_spread,waveform_velocity_above,nn_miss_rate,isolation_distance,waveform_repolarization_slope,silhouette_score,max_drift,isi_violations,waveform_PT_ratio,waveform_halfwidth,amplitude_cutoff,presence_ratio,waveform_amplitude,L_ratio,nn_hit_rate,cluster_id,peak_channel_id,waveform_recovery_slope,waveform_duration,snr,cumulative_drift,local_index_unit,d_prime,firing_rate,waveform_velocity_below,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951781164,90.0,0.824121,0.050647,63.568084,0.208775,0.122490,11.31,0.107623,0.397729,0.274707,0.080107,0.99,74.728680,0.036502,0.854667,6,849976420,-0.037742,0.576884,2.198320,102.08,6,3.184190,14.848563,-0.206030,NaN,300.0,3.319143,1.207018,0.951667,1.748148,1.024631,0.284343,0.012586,0.034406,0.019169,20.000,0.821287,0.516,0.04,1.0,0.444643,0.454545,0.252174,NaN,0.346154,225.0,90.0,0.001058,0.000762,0.331550,NaN,4.022038e-03,-30.000,4983,0.25,12.825823,14.787745,12.106041,28.394879,13.650431,NaN,0.2335,False,False,False,False,False,0.023542,False,0.006183,0.001460,0.007285,0.009112,0.007424,900,11,792676154,89,484682470.0,ProS,8017.0,1857.0,7179.0,probeA,See electrode locations,29999.964439,1249.998518,True
951781179,60.0,0.961474,0.008094,75.035952,0.245926,0.217450,5.66,0.049019,0.404661,0.247236,0.011366,0.99,95.422080,0.008291,0.974667,9,849976428,-0.049703,0.604355,2.627821,77.17,9,4.840396,11.982257,-1.716918,NaN,300.0,4.092459,7.583198,1.181772,12.293939,0.692177,0.269200,0.052923,0.018655,0.039166,90.000,1.038834,0.709,0.04,2.0,0.405357,0.617391,0.467630,1.0,0.250000,225.0,120.0,0.013629,0.046922,0.009657,0.088788,1.621455e-02,-20.000,4916,0.00,13.234059,18.304831,10.136618,5.254412,13.298063,NaN,0.0095,False,False,False,False,False,0.003729,False,0.004917,0.000106,0.007312,0.077078,0.005563,940,11,792676154,93,382.0,CA1,8010.0,1824.0,7199.0,probeA,See electrode locations,29999.964439,1249.998518,True
951781341,100.0,0.627901,0.006529,86.322527,0.089462,0.108749,87.16,0.031364,0.594550,0.178559,0.001880,0.99,42.931785,0.008108,0.936667,48,849976628,-0.014602,0.755444,1.071927,315.04,48,2.944735,4.698033,0.068677,NaN,100.0,2.505263,2.156012,2.928142,3.175758,3.807692,0.819604,0.006309,0.148823,0.176272,50.000,0.207438,0.201,0.02,15.0,NaN,0.680952,0.370751,NaN,0.696203,45.0,120.0,NaN,0.000006,0.166283,NaN,3.304738e-07,-30.000,5017,0.00,2.144432,6.821014,9.953916,0.538380,7.790731,NaN,0.1265,False,False,False,False,False,0.298831,True,0.287501,0.622035,0.120567,0.497674,0.238304,1940,11,792676154,193,394.0,VISam,7814.0,890.0,7491.0,probeA,See electrode locations,29999.964439,1249.998518,True
951781346,110.0,0.997448,0.000651,74.737358,0.239747,0.202850,114.30,0.000000,0.338191,0.164824,0.000154,0.99,107.153280,0.012747,0.996667,49,849976634,-0.030542,0.563149,2.684842,144.72,49,4.099752,7.144577,0.068677,NaN,1100.0,4.425571,0.921290,1.069148,0.831008,1.069023,0.293031,0.003705,0.169407,0.047248,29.091,1.318424,0.000,0.02,1.0,NaN,0.206349,0.394410,NaN,0.201452,270

In [10]:
V1_units = units[(units.ecephys_structure_acronym == 'VISp') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 26


,waveform_spread,waveform_velocity_above,nn_miss_rate,isolation_distance,waveform_repolarization_slope,silhouette_score,max_drift,isi_violations,waveform_PT_ratio,waveform_halfwidth,amplitude_cutoff,presence_ratio,waveform_amplitude,L_ratio,nn_hit_rate,cluster_id,peak_channel_id,waveform_recovery_slope,waveform_duration,snr,cumulative_drift,local_index_unit,d_prime,firing_rate,waveform_velocity_below,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951785427,110.0,1.359461,0.003642,50.542769,0.107568,0.093842,64.62,0.082598,0.495986,0.206030,0.000254,0.97,39.382395,0.016098,0.905128,100,849978602,-0.025759,0.645561,2.517883,191.93,93,3.394125,2.363741,2.060302e-01,NaN,200.0,1.897564,1.626667,0.800000,1.112821,1.026779,0.365789,0.036898,0.757609,0.633587,50.000,1.085672,0.0,0.08,4.0,NaN,1.000000,-0.459770,NaN,0.324379,90.0,90.0,NaN,0.191054,0.022014,NaN,5.599960e-05,15.000,4937,0.50,2.894719,0.079934,2.265643,0.294958,6.644369,NaN,0.1275,False,False,False,False,False,0.710475,False,0.652396,1.000000,0.378709,0.838474,0.485745,1860,43,792676158,184,385.0,VISp,8957.0,1415.0,8001.0,probeC,See electrode locations,29999.995177,1249.999799,True
951785584,100.0,0.240369,0.001361,89.412808,0.157824,0.141122,93.13,0.001270,0.479723,0.206030,0.000747,0.99,192.133305,0.000276,0.985152,132,849978636,-0.029214,0.480737,1.702155,111.08,125,3.945473,7.782070,-6.867672e-01,NaN,400.0,4.548454,1.034203,1.070210,1.308772,2.134526,0.474176,0.049982,0.199927,0.078735,45.000,0.040348,0.0,0.02,1.0,NaN,0.428571,0.386822,NaN,0.601370,90.0,90.0,NaN,0.071029,0.002442,NaN,6.435085e-10,-5.000,4938,0.00,3.794321,4.662803,12.465401,1.712728,10.573705,NaN,0.1265,False,False,False,False,False,0.145576,False,0.320426,0.005276,0.074134,0.296822,0.174678,2020,11,792676158,201,385.0,VISp,9022.0,1219.0,8012.0,probeC,See electrode locations,29999.995177,1249.999799,True
951785569,100.0,0.171692,0.000476,76.571029,0.417232,0.049238,88.64,0.004685,0.528142,0.164824,0.004078,0.99,113.469330,0.001330,0.984000,129,849978638,-0.092448,0.219765,3.068335,155.33,122,5.534640,12.154944,-1.373534e-01,NaN,600.0,5.851572,1.186056,3.228203,1.989744,2.398167,0.355097,0.067931,0.172360,0.074323,50.000,4.029339,0.0,0.02,1.0,NaN,-0.081839,0.348764,-0.487179,0.396791,270.0,90.0,NaN,0.594328,0.140239,0.038274,2.888113e-04,15.000,4963,0.75,8.028119,7.886797,27.993080,2.160099,13.887564,NaN,0.0915,False,False,False,False,False,0.425780,False,0.218840,0.752939,0.125231,0.439695,0.132190,2040,59,792676158,202,385.0,VISp,9025.0,1210.0,8013.0,probeC,See electrode locations,29999.995177,1249.999799,True
951785620,140.0,0.961474,0.001879,50.645587,0.244915,0.046139,135.87,0.001125,0.487884,0.247236,0.000270,0.99,117.302250,0.027829,0.978000,139,849978648,-0.043436,0.521943,2.318700,168.74,132,2.688601,14.318910,2.354630e-01,NaN,1100.0,1.164195,0.592157,1.136381,0.571429,0.846235,0.220533,0.018285,0.097637,0.023138,49.091,0.454873,0.0,0.04,1.0,NaN,0.360195,0.393435,NaN,0.374224,0.0

## Find average firing rate for static gratings. 

In [11]:
print(V1_units.firing_rate_sg)

unit_id
951785427     6.644369
951785584    10.573705
951785569    13.887564
951785620    17.164120
951785595     4.129833
951785683     3.805442
951785629     7.475664
951785671    10.980694
951785667     6.980750
951785658     3.347829
951785654    17.542466
951785643    12.031137
951785692    11.301089
951785719     6.601738
951785714    10.020841
951785755    11.710076
951785709     8.127778
951785745    13.114219
951785739    10.684278
951785730    33.515596
951785788     2.096890
951785784     3.056743
951785842     2.273407
951785860     8.830516
951786038    17.874185
951786065    13.945515
Name: firing_rate_sg, dtype: float64


In [12]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

10.296786294128555


## Find average firing rate for drifting gratings. 

In [13]:
print(V1_units.firing_rate_dg)

unit_id
951785427     2.894719
951785584     3.794321
951785569     8.028119
951785620    10.122779
951785595     4.018825
951785683     3.715427
951785629     3.103338
951785671     9.461447
951785667     7.197350
951785658     5.169935
951785654     9.503277
951785643    10.280567
951785692     8.262683
951785719     6.623383
951785714     5.737018
951785755     9.183465
951785709     8.820235
951785745    10.180494
951785739     8.442709
951785730    19.009730
951785788     1.531283
951785784     3.125576
951785842     1.918340
951785860     4.430768
951786038    19.499508
951786065     5.695188
Name: firing_rate_dg, dtype: float64


In [14]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

7.298095593794236


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])